# CFP Resume Analysis - Team Sheets

This notebook provides committee-style resume analysis for all teams, focusing on:
- Strength of Record (SOR) - quality of wins given schedule
- Strength of Schedule (SOS) - difficulty of opponents faced
- Quality Wins - wins vs top-tier opponents
- Bad Losses - losses to lower-ranked teams
- Conference championship status
- Resume vs Predictive comparison

This separates **what teams have accomplished** (resume) from **how good they are** (predictive power).

In [1]:
# Cell 1: Setup and Imports
import sys
import os
import pandas as pd
import numpy as np
import json
from pathlib import Path

# Add src to path
sys.path.insert(0, os.path.abspath('..'))

from src.utils.metrics import (
    calculate_sor,
    calculate_sos,
    calculate_quality_wins,
    identify_bad_losses,
    build_resume_dataframe,
    compare_resume_vs_predictive
)

# Create output directories
output_dir = Path('./data/output')
rankings_dir = output_dir / 'rankings'
exports_dir = output_dir / 'exports'
rankings_dir.mkdir(parents=True, exist_ok=True)
exports_dir.mkdir(parents=True, exist_ok=True)

print("✅ Imports loaded successfully")
print(f"Output directory: {output_dir}")

✅ Imports loaded successfully
Output directory: data/output


In [2]:
# Cell 2: Load Previous Results
year = 2025
week = 15
# Load composite rankings from previous notebook
final_rankings = pd.read_csv(rankings_dir / f'composite_rankings_{year}_week{week}.csv')

# Load games data for resume analysis
cache_dir = Path(f'./data/cache/{year}')
parquet_path = cache_dir / f'games_w{week}.parquet'
csv_path = cache_dir / f'games_w{week}.csv'

if parquet_path.exists():
    try:
        games_df = pd.read_parquet(parquet_path)
    except (ImportError, ModuleNotFoundError):
        games_df = pd.read_csv(csv_path)
elif csv_path.exists():
    games_df = pd.read_csv(csv_path)
else:
    raise FileNotFoundError("No cached data found. Run 01_data_pipeline.ipynb first.")

# Load individual rankings for resume vs predictive split
colley_rankings = pd.read_csv(rankings_dir / f'colley_rankings_{year}_week{week}.csv')
massey_rankings = pd.read_csv(rankings_dir / f'massey_rankings_{year}_week{week}.csv')
elo_rankings = pd.read_csv(rankings_dir / f'elo_rankings_{year}_week{week}.csv')

print(f"✅ Loaded {len(final_rankings)} teams")
print(f"✅ Loaded {len(games_df)} games")
print(f"Season: {year}, Week: {week}")


✅ Loaded 136 teams
✅ Loaded 557 games
Season: 2025, Week: 15


---

## Strength of Record (SOR)

**Definition:** The probability that an average Top-25 team would achieve at least this team's record against this exact schedule.

**Why it matters:** SOR evaluates **how impressive** a team's record is given who they played. A 10-2 team with a brutal schedule can have better SOR than an 11-1 team with weak opponents.

**Calculation:** Uses game-by-game win probabilities for an average Top-25 team (baseline rating ~0.75) against each opponent, then calculates the probability of achieving the observed number of wins.

In [3]:
# Cell 3: Calculate Strength of Record (SOR)
from scipy.stats import norm
from sklearn.preprocessing import MinMaxScaler

# Build opponent ratings dictionary from PREDICTIVE scores (not composite!)
# Use predictive rankings to represent team strength for SOR calculation
scaler = MinMaxScaler()

# Option 1: Use the predictive_score from loaded rankings
if 'predictive_score' in final_rankings.columns:
    # Normalize predictive scores to 0-1
    final_rankings['normalized_predictive'] = scaler.fit_transform(
        final_rankings[['predictive_score']]
    )
    opponent_ratings = dict(zip(
        final_rankings['team'],
        final_rankings['normalized_predictive']
    ))
    print("✅ Using predictive_score for opponent ratings")

# Option 2: Calculate from Massey + Elo if predictive_score not available
else:
    # Combine Massey and Elo
    massey_dict = dict(zip(massey_rankings['team'], massey_rankings['rating']))
    elo_dict = dict(zip(elo_rankings['team'], elo_rankings['rating']))
    
    # Average them and normalize
    combined_ratings = {}
    for team in final_rankings['team']:
        massey_norm = scaler.fit_transform([[massey_dict.get(team, 0)]])[0][0]
        elo_norm = scaler.fit_transform([[elo_dict.get(team, 0)]])[0][0]
        combined_ratings[team] = (massey_norm + elo_norm) / 2
    
    opponent_ratings = combined_ratings
    print("✅ Using Massey + Elo for opponent ratings")

print(f"Opponent ratings range: {min(opponent_ratings.values()):.4f} to {max(opponent_ratings.values()):.4f}")

# Calculate SOR for each team
sor_scores = {}

for _, team_row in final_rankings.iterrows():
    team = team_row['team']
    
    # Get team's games
    team_games = games_df[
        (games_df['home_team'] == team) | (games_df['away_team'] == team)
    ]
    
    # Build opponent ratings list
    opp_ratings_list = []
    for _, game in team_games.iterrows():
        opponent = game['away_team'] if game['home_team'] == team else game['home_team']
        opp_rating = opponent_ratings.get(opponent, 0.5)
        opp_ratings_list.append(opp_rating)
    
    # Calculate SOR
    team_record = {
        'wins': int(team_row['wins']),
        'losses': int(team_row['losses'])
    }
    
    sor_score = calculate_sor(
        team_record=team_record,
        opponent_ratings=opp_ratings_list,
        baseline_rating=0.75,
        rating_scale=0.25
    )
    
    sor_scores[team] = sor_score

# Rank SOR scores (higher SOR score = better record given schedule)
sor_series = pd.Series(sor_scores)
sor_ranks = sor_series.rank(method='min', ascending=False).to_dict()

print("\n✅ SOR calculated for all teams")
print(f"SOR range: {sor_series.min():.4f} to {sor_series.max():.4f}")

print("\nTop 10 Teams by SOR (best records given schedule):")
top_sor = sor_series.sort_values(ascending=False).head(10)
for team, score in top_sor.items():
    rank = int(sor_ranks[team])
    wins = final_rankings[final_rankings['team'] == team]['wins'].values[0]
    losses = final_rankings[final_rankings['team'] == team]['losses'].values[0]
    print(f"  {rank:2d}. {team:<25} {wins}-{losses}  SOR={score:.4f}")

print("\nBottom 10 Teams by SOR (worst records given schedule):")
bottom_sor = sor_series.sort_values().head(10)
for team, score in bottom_sor.items():
    rank = int(sor_ranks[team])
    wins = final_rankings[final_rankings['team'] == team]['wins'].values[0]
    losses = final_rankings[final_rankings['team'] == team]['losses'].values[0]
    print(f"  {rank:2d}. {team:<25} {wins}-{losses}  SOR={score:.4f}")

✅ Using predictive_score for opponent ratings
Opponent ratings range: 0.0000 to 1.0000

✅ SOR calculated for all teams
SOR range: -0.0000 to 0.9422

Top 10 Teams by SOR (best records given schedule):
   1. Notre Dame                9-0  SOR=0.9422
   2. James Madison             9-0  SOR=0.8913
   3. Ole Miss                  6-1  SOR=0.5747
   4. Alabama                   7-1  SOR=0.4710
   5. Indiana                   8-0  SOR=0.4058
   6. Texas A&M                 7-1  SOR=0.3911
   7. Ohio State                9-0  SOR=0.3557
   8. Navy                      6-2  SOR=0.2456
   9. BYU                       8-1  SOR=0.2317
  10. Virginia                  7-1  SOR=0.2147

Bottom 10 Teams by SOR (worst records given schedule):
  129. Massachusetts             0-9  SOR=-0.0000
  129. Maryland                  0-8  SOR=-0.0000
  129. Georgia State             0-8  SOR=-0.0000
  129. Charlotte                 0-8  SOR=-0.0000
  129. Oklahoma State            0-9  SOR=-0.0000
  129. Arkansa

---

## Strength of Schedule (SOS)

**Definition:** Average quality of opponents faced, accounting for both:
1. Opponents' records (direct strength)
2. Opponents' opponents' records (OOR - prevents inflated opponent records)

**Why it matters:** Prevents rewarding teams whose opponents only looked good because they beat weak teams.

**Calculation:** Weighted combination of opponents' win percentage (66.7%) and opponents' opponents' win percentage (33.3%).

In [4]:
# Cell 4: Calculate Strength of Schedule (SOS) using Win Percentages
# Formula: SOS = (2*OR + OOR)/3 where OR and OOR are win percentages
def calculate_team_sos(team, games_df, include_oor=True, oor_weight=0.33):
    """
    Calculate Strength of Schedule using metrics module.
    Includes opponent's opponents' records (OOR) to prevent inflated schedules.
    Formula: SOS = (2*OR + OOR)/3 where OR and OOR are win percentages
    """
    team_games = games_df[(games_df['home_team'] == team) | (games_df['away_team'] == team)]
    
    opponents = []
    for _, game in team_games.iterrows():
        opponent = game['away_team'] if game['home_team'] == team else game['home_team']
        opponents.append(opponent)
    
    # Get opponent records
    opponents_records = []
    opponents_opp_records = []
    
    for opp in opponents:
        opp_games = games_df[(games_df['home_team'] == opp) | (games_df['away_team'] == opp)]
        
        opp_wins = 0
        opp_losses = 0
        
        # Get opponent's record - EXCLUDE the game against the original team
        # This prevents circular dependency (team beating opponent makes opponent look worse)
        for _, g in opp_games.iterrows():
            # Skip the game between this team and the opponent
            if (g['home_team'] == team and g['away_team'] == opp) or \
               (g['home_team'] == opp and g['away_team'] == team):
                continue
            
            if g['home_team'] == opp:
                if g['home_score'] > g['away_score']:
                    opp_wins += 1
                else:
                    opp_losses += 1
            else:
                if g['away_score'] > g['home_score']:
                    opp_wins += 1
                else:
                    opp_losses += 1
        
        opponents_records.append((opp_wins, opp_losses))
        
        # Get opponent's opponents' records (for OOR calculation)
        opp_opp_records = []
        for _, g in opp_games.iterrows():
            opp_opp = g['away_team'] if g['home_team'] == opp else g['home_team']
            
            # Skip if opponent's opponent is the original team (circular)
            if opp_opp == team:
                continue
            
            opp_opp_games = games_df[
                (games_df['home_team'] == opp_opp) | (games_df['away_team'] == opp_opp)
            ]
            
            opp_opp_wins = 0
            opp_opp_losses = 0
            
            for _, gg in opp_opp_games.iterrows():
                if gg['home_team'] == opp_opp:
                    if gg['home_score'] > gg['away_score']:
                        opp_opp_wins += 1
                    else:
                        opp_opp_losses += 1
                else:
                    if gg['away_score'] > gg['home_score']:
                        opp_opp_wins += 1
                    else:
                        opp_opp_losses += 1
            
            opp_opp_records.append((opp_opp_wins, opp_opp_losses))
        
        opponents_opp_records.append(opp_opp_records)
    
    # Use metrics module SOS calculation
    sos_score = calculate_sos(
        opponents_records=opponents_records,
        opponents_opp_records=opponents_opp_records,
        include_oor=include_oor,
        oor_weight=oor_weight
    )
    
    return sos_score

# Calculate SOS for each team using win percentages
sos_scores = {}
for team in final_rankings['team']:
    sos_score = calculate_team_sos(team, games_df, include_oor=True, oor_weight=0.33)
    sos_scores[team] = sos_score

# Rank SOS scores (higher SOS = tougher schedule)
sos_series = pd.Series(sos_scores)
sos_ranks = sos_series.rank(method='min', ascending=False).to_dict()

print("✅ SOS calculated for all teams using win percentages")
print(f"   Formula: SOS = (2*OR + OOR)/3")
print(f"   Where OR = Opponents' win%, OOR = Opponents' opponents' win%")
print(f"SOS range: {sos_series.min():.3f} to {sos_series.max():.3f}")

print("\nTop 10 Toughest Schedules:")
top_sos = sos_series.sort_values(ascending=False).head(10)
for team, score in top_sos.items():
    rank = int(sos_ranks[team])
    wins = final_rankings[final_rankings['team'] == team]['wins'].values[0]
    losses = final_rankings[final_rankings['team'] == team]['losses'].values[0]
    print(f"  {rank:2d}. {team:<25} {wins}-{losses}  SOS={score:.3f}")

print("\nTop 10 Easiest Schedules:")
bottom_sos = sos_series.sort_values().head(10)
for team, score in bottom_sos.items():
    rank = int(sos_ranks[team])
    wins = final_rankings[final_rankings['team'] == team]['wins'].values[0]
    losses = final_rankings[final_rankings['team'] == team]['losses'].values[0]
    print(f"  {rank:2d}. {team:<25} {wins}-{losses}  SOS={score:.3f}")


✅ SOS calculated for all teams using win percentages
   Formula: SOS = (2*OR + OOR)/3
   Where OR = Opponents' win%, OOR = Opponents' opponents' win%
SOS range: 0.390 to 0.653

Top 10 Toughest Schedules:
   1. Wisconsin                 2-6  SOS=0.653
   2. Charlotte                 0-8  SOS=0.618
   3. USC                       5-3  SOS=0.607
   4. South Carolina            2-6  SOS=0.596
   5. LSU                       3-5  SOS=0.590
   6. Oklahoma                  6-2  SOS=0.585
   7. Colorado                  1-7  SOS=0.575
   8. Purdue                    0-8  SOS=0.570
   9. West Virginia             2-6  SOS=0.569
  10. Rice                      2-6  SOS=0.568

Top 10 Easiest Schedules:
  136. Sam Houston               2-6  SOS=0.390
  135. Texas A&M                 7-1  SOS=0.402
  134. Indiana                   8-0  SOS=0.413
  133. Fresno State              4-3  SOS=0.420
  132. Nebraska                  4-4  SOS=0.429
  131. Hawai'i                   5-2  SOS=0.439
  130. Dela

---

## Quality Wins and Bad Losses

**Quality Wins:** Wins against high-ranked opponents demonstrate ability to beat elite competition.
- Top 5 wins: Elite wins
- Top 12 wins: Playoff-caliber wins
- Top 25 wins: Quality wins
- **Road/Neutral Quality Wins:** The committee values wins against Top 25 teams away from home significantly more than home wins. These are tracked separately as a key differentiator.

**Bad Losses:** Losses to teams ranked outside Top 25 hurt resume significantly.

**FCS Handling:** Games against lower-division (FCS) opponents are treated as rank 999 and do not count toward quality wins, ensuring accurate resume evaluation.

**Evaluation Method:** Opponents are evaluated at their **current strength** (from final rankings) rather than at-the-time-played rank, aligning with the committee's "body of work" philosophy.

In [5]:
# Cell 5: Calculate Quality Wins, Bad Losses, and Road Context
# Enhanced with road/neutral quality wins tracking per CFP Committee protocols
quality_wins_data = {}
bad_losses_data = {}
vs_top25_data = {}
road_top25_wins_data = {}  # NEW: Track road/neutral ranked wins (key differentiator)

# Create a lookup for quick rank access (evaluates opponents at current strength)
team_rank_dict = dict(zip(final_rankings['team'], final_rankings['rank']))

for team in final_rankings['team']:
    team_games = games_df[
        (games_df['home_team'] == team) | (games_df['away_team'] == team)
    ]
    
    win_opponents = []
    loss_opponents = []
    road_ranked_wins = 0  # Counter for road/neutral quality wins
    
    for _, game in team_games.iterrows():
        # Determine if neutral site (critical for CFP evaluation)
        is_neutral = game.get('neutral_site', False)
        
        if game['home_team'] == team:
            opponent = game['away_team']
            won = game['home_score'] > game['away_score']
            is_road_win = False  # Home win
        else:
            opponent = game['home_team']
            won = game['away_score'] > game['home_score']
            is_road_win = True  # True road game
            
        opp_rank = team_rank_dict.get(opponent, 999)  # FCS/unranked teams = rank 999
        
        if won:
            win_opponents.append(opponent)
            # Check for High Quality Road/Neutral Win (Committee Differentiator)
            # The committee values wins against Top 25 teams away from home significantly more
            if opp_rank <= 25 and (is_road_win or is_neutral):
                road_ranked_wins += 1
        else:
            loss_opponents.append(opponent)
    
    # Calculate quality wins (using existing function)
    win_opp_ranks = [team_rank_dict.get(opp, 999) for opp in win_opponents]
    qw = calculate_quality_wins(
        opponent_ranks=win_opp_ranks,
        thresholds={'top_5': 5, 'top_12': 12, 'top_25': 25}
    )
    quality_wins_data[team] = qw
    road_top25_wins_data[team] = road_ranked_wins  # Store road/neutral quality wins
    
    # Calculate bad losses (losses to teams outside Top 25)
    loss_opp_ranks = [team_rank_dict.get(opp, 999) for opp in loss_opponents]
    bl_count, bl_ranks = identify_bad_losses(
        loss_opponent_ranks=loss_opp_ranks,
        threshold=25
    )
    bad_losses_data[team] = bl_count
    
    # Format Top 25 Record string (e.g., "3-1")
    top25_wins = qw['top_25']
    top25_losses = sum(1 for rank in loss_opp_ranks if rank <= 25)
    vs_top25_data[team] = f"{top25_wins}-{top25_losses}"

print("✅ Quality wins, road value, and bad losses calculated")
print("\nTeams with Most Top 25 Wins:")
top25_wins_sorted = sorted(
    [(team, qw['top_25']) for team, qw in quality_wins_data.items()],
    key=lambda x: x[1],
    reverse=True
)[:10]
for team, count in top25_wins_sorted:
    print(f"  {team:<25} {count} wins vs Top 25")

print("\nTeams with Most Road/Neutral Top 25 Wins (Committee Differentiator):")
road_top25_sorted = sorted(
    [(team, count) for team, count in road_top25_wins_data.items()],
    key=lambda x: x[1],
    reverse=True
)[:10]
for team, count in road_top25_sorted:
    if count > 0:
        print(f"  {team:<25} {count} road/neutral wins vs Top 25")


✅ Quality wins, road value, and bad losses calculated

Teams with Most Top 25 Wins:
  Notre Dame                2 wins vs Top 25
  Alabama                   2 wins vs Top 25
  Georgia                   2 wins vs Top 25
  Ole Miss                  2 wins vs Top 25
  Texas                     2 wins vs Top 25
  Wake Forest               2 wins vs Top 25
  James Madison             1 wins vs Top 25
  Oregon                    1 wins vs Top 25
  Indiana                   1 wins vs Top 25
  Texas Tech                1 wins vs Top 25

Teams with Most Road/Neutral Top 25 Wins (Committee Differentiator):
  Alabama                   1 road/neutral wins vs Top 25
  Indiana                   1 road/neutral wins vs Top 25
  Ole Miss                  1 road/neutral wins vs Top 25
  South Florida             1 road/neutral wins vs Top 25
  Oklahoma                  1 road/neutral wins vs Top 25
  Texas                     1 road/neutral wins vs Top 25
  Toledo                    1 road/neutral wins 

---

## Conference Champions

Conference championship status is critical for CFP selection (automatic bid consideration).

**Simulation Logic:**
1. **Identify Participants:** Top 2 teams in each conference based on:
   - Conference win percentage (primary)
   - Overall record (secondary)
   - Composite rank (tertiary tiebreaker)
2. **Simulate Championship Game:** Uses predictive scores (Massey + Elo) to determine winner
3. **Allow Upsets:** Lower-ranked teams can win if they have similar predictive ratings
4. **Skip Independents:** Teams without conferences cannot be conference champions

This ensures accurate playoff selection per the 5+7 format, where the 5 highest-ranked conference champions receive automatic bids.

In [6]:
# Cell 6: Simulate Conference Championship Games
# Extract conference information from games data
team_conferences = {}
for _, game in games_df.iterrows():
    if pd.notna(game.get('home_conference')):
        team_conferences[game['home_team']] = game['home_conference']
    if pd.notna(game.get('away_conference')):
        team_conferences[game['away_team']] = game['away_conference']

conf_champions = {}
championship_games_found = False

# Check if final_rankings has conference and conf_champ columns (manual input)
if 'conference' in final_rankings.columns and 'conf_champ' in final_rankings.columns:
    champs_df = final_rankings[final_rankings['conf_champ'].str.contains('Yes', na=False)]
    if not champs_df.empty:
        championship_games_found = True
        for _, row in champs_df.iterrows():
            conf = row['conference']
            if pd.notna(conf):
                conf_champions[conf] = row['team']
        print("✅ Conference champions found in rankings data")

# If not in rankings, check for scheduled or played championship games in games data
if not championship_games_found:
    # First, try to fetch scheduled championship games (may not have scores yet)
    # We need to check the raw API data for scheduled games
    try:
        import os
        from dotenv import load_dotenv
        import requests
        
        load_dotenv()
        api_key = os.getenv('CFBD_API_KEY')
        if api_key:
            api_key = api_key.strip().strip('"').strip("'")
            
            # Fetch week 15 games (championship week) - include games without scores
            base_url = "https://api.collegefootballdata.com/games"
            headers = {
                "Authorization": f"Bearer {api_key}",
                "accept": "application/json"
            }
            
            # Try both regular and postseason season types
            scheduled_championship_games = []
            for season_type in ["regular", "postseason"]:
                params = {
                    "year": year,
                    "week": 15,
                    "seasonType": season_type,
                    "division": "fbs"
                }
                
                try:
                    response = requests.get(base_url, headers=headers, params=params, timeout=5)
                    if response.status_code == 200:
                        week_games = response.json()
                        scheduled_championship_games.extend(week_games)
                except:
                    continue
            
            # Process scheduled games - look for same-conference, neutral site games
            if scheduled_championship_games:
                champ_matchups = {}
                
                for game in scheduled_championship_games:
                    home_team = game.get('home_team') or game.get('homeTeam')
                    away_team = game.get('away_team') or game.get('awayTeam')
                    home_conf = game.get('home_conference') or game.get('homeConference')
                    away_conf = game.get('away_conference') or game.get('awayConference')
                    neutral = game.get('neutral_site') or game.get('neutralSite', False)
                    home_score = game.get('home_points') or game.get('homePoints')
                    away_score = game.get('away_points') or game.get('awayPoints')
                    
                    # Identify championship games: same conference, neutral site (or week 15)
                    if (home_conf and away_conf and home_conf == away_conf and
                        (neutral or game.get('week') == 15)):
                        
                        conf = home_conf
                        # Check if game is already played (has scores)
                        if home_score is not None and away_score is not None:
                            # Game is complete - use result
                            if conf not in conf_champions:
                                if int(home_score) > int(away_score):
                                    conf_champions[conf] = home_team
                                else:
                                    conf_champions[conf] = away_team
                                championship_games_found = True
                        else:
                            # Game is scheduled but not played - store matchup for simulation
                            if conf not in champ_matchups:
                                champ_matchups[conf] = {
                                    'team_a': home_team,
                                    'team_b': away_team,
                                    'neutral': neutral
                                }
                
                if championship_games_found:
                    print("✅ Conference champions identified from completed championship games")
                
                # Simulate scheduled games that haven't been played
                if champ_matchups:
                    print("🎮 Simulating scheduled conference championship games...")
                    print()
                    
                    # Get predictive scores for simulation
                    if 'predictive_score' in final_rankings.columns:
                        predictive_scores = dict(zip(final_rankings['team'], final_rankings['predictive_score']))
                    else:
                        # Calculate from Massey + Elo if not available
                        massey_dict = dict(zip(massey_rankings['team'], massey_rankings['rating']))
                        elo_dict = dict(zip(elo_rankings['team'], elo_rankings['rating']))
                        from sklearn.preprocessing import MinMaxScaler
                        scaler = MinMaxScaler()
                        
                        all_ratings = list(massey_dict.values()) + list(elo_dict.values())
                        scaler.fit([[r] for r in all_ratings])
                        
                        predictive_scores = {}
                        for team in final_rankings['team']:
                            massey_norm = scaler.transform([[massey_dict.get(team, 0)]])[0][0]
                            elo_norm = scaler.transform([[elo_dict.get(team, 0)]])[0][0]
                            predictive_scores[team] = 0.5 * massey_norm + 0.5 * elo_norm
                    
                    # Simulate each scheduled matchup
                    for conf, matchup in champ_matchups.items():
                        team_a = matchup['team_a']
                        team_b = matchup['team_b']
                        
                        # Get team ranks and predictive scores
                        team_a_row = final_rankings[final_rankings['team'] == team_a]
                        team_b_row = final_rankings[final_rankings['team'] == team_b]
                        
                        if team_a_row.empty or team_b_row.empty:
                            continue
                        
                        team_a_rank = team_a_row.iloc[0]['rank']
                        team_b_rank = team_b_row.iloc[0]['rank']
                        team_a_score = predictive_scores.get(team_a, 0.0)
                        team_b_score = predictive_scores.get(team_b, 0.0)
                        
                        # Simulate the game
                        # Higher-ranked team gets slight advantage (neutral site)
                        hfa_adjustment = 0.01 if team_a_rank < team_b_rank else -0.01
                        spread = (team_a_score - team_b_score) + hfa_adjustment
                        
                        # Deterministic: higher score wins
                        if spread > 0:
                            winner = team_a
                            upset = ""
                        else:
                            winner = team_b
                            upset = " ⚠️ UPSET!" if team_b_rank > team_a_rank else ""
                        
                        conf_champions[conf] = winner
                        print(f"  {conf:<25} #{team_a_rank} {team_a:<20} vs #{team_b_rank} {team_b:<20} → {winner}{upset}")
                    
                    if champ_matchups:
                        print()
                        championship_games_found = True
                        
    except Exception as e:
        # If API fetch fails, fall through to backup method
        pass
    
    # Fallback: Check for completed championship games in existing games_df
    if not championship_games_found:
        # Look for completed championship games: neutral site, week 15, same conference
        potential_championship_games = games_df[
            (games_df['week'] == 15) &  # Championship Week
            (games_df['neutral_site'] == True) &  # Championship games are usually neutral site
            (games_df['home_conference'] == games_df['away_conference']) &
            (games_df['home_conference'].notna())
        ]
        
        if not potential_championship_games.empty:
            # Use actual championship game results
            for conf in potential_championship_games['home_conference'].unique():
                conf_championship_games = potential_championship_games[
                    potential_championship_games['home_conference'] == conf
                ].sort_values('date', ascending=False)
                
                if not conf_championship_games.empty:
                    # Use the latest game as the championship game
                    champ_game = conf_championship_games.iloc[0]
                    if champ_game['home_score'] > champ_game['away_score']:
                        conf_champions[conf] = champ_game['home_team']
                    else:
                        conf_champions[conf] = champ_game['away_team']
                    championship_games_found = True
            
            if championship_games_found:
                print("✅ Conference champions identified from championship games in cached data")
                print(f"   Found {len(potential_championship_games)} championship games")

# Fallback: Simulate conference championship games if scheduled games not found
# This identifies top 2 teams per conference and simulates a matchup
if not championship_games_found:
    print("🏆 Simulating Conference Championship Games...")
    print("   (Scheduled games not found - using top 2 teams per conference)")
    print()
    
    # Helper function to calculate conference record
    def calculate_conference_record(team, conference, games_df, team_conferences):
        """Calculate conference wins and losses for a team (only games vs same-conference opponents)"""
        conf_wins = 0
        conf_losses = 0
        
        # Get all games for this team
        team_games = games_df[
            (games_df['home_team'] == team) | (games_df['away_team'] == team)
        ]
        
        for _, game in team_games.iterrows():
            # Determine opponent
            if game['home_team'] == team:
                opponent = game['away_team']
                # Check if opponent is in same conference
                opp_conf = team_conferences.get(opponent)
                if opp_conf != conference:
                    continue
                # Count win/loss
                if game['home_score'] > game['away_score']:
                    conf_wins += 1
                else:
                    conf_losses += 1
            else:  # away_team == team
                opponent = game['home_team']
                # Check if opponent is in same conference
                opp_conf = team_conferences.get(opponent)
                if opp_conf != conference:
                    continue
                # Count win/loss
                if game['away_score'] > game['home_score']:
                    conf_wins += 1
                else:
                    conf_losses += 1
        
        total = conf_wins + conf_losses
        conf_win_pct = conf_wins / total if total > 0 else 0.0
        
        return conf_wins, conf_losses, conf_win_pct
    
    # Group teams by conference
    conferences_dict = {}
    for team in final_rankings['team']:
        conf = team_conferences.get(team)
        if conf and pd.notna(conf):
            if conf not in conferences_dict:
                conferences_dict[conf] = []
            conferences_dict[conf].append(team)
    
    # Get predictive scores for simulation
    if 'predictive_score' in final_rankings.columns:
        predictive_scores = dict(zip(final_rankings['team'], final_rankings['predictive_score']))
    else:
        # Calculate from Massey + Elo if not available
        massey_dict = dict(zip(massey_rankings['team'], massey_rankings['rating']))
        elo_dict = dict(zip(elo_rankings['team'], elo_rankings['rating']))
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        
        all_ratings = list(massey_dict.values()) + list(elo_dict.values())
        scaler.fit([[r] for r in all_ratings])
        
        predictive_scores = {}
        for team in final_rankings['team']:
            massey_norm = scaler.transform([[massey_dict.get(team, 0)]])[0][0]
            elo_norm = scaler.transform([[elo_dict.get(team, 0)]])[0][0]
            predictive_scores[team] = 0.5 * massey_norm + 0.5 * elo_norm
    
    # Simulate championship games for each conference
    for conf, teams in conferences_dict.items():
        # Skip independents - they can't have conference champions
        if conf in ['FBS Independents', 'Independent']:
            continue
        
        conf_teams_df = final_rankings[final_rankings['team'].isin(teams)].copy()
        
        if conf_teams_df.empty:
            continue
        
        # Calculate conference standings (conference win %)
        conf_standings = []
        for _, row in conf_teams_df.iterrows():
            team = row['team']
            conf_wins, conf_losses, conf_win_pct = calculate_conference_record(team, conf, games_df, team_conferences)
            conf_standings.append({
                'team': team,
                'conf_wins': conf_wins,
                'conf_losses': conf_losses,
                'conf_win_pct': conf_win_pct,
                'overall_wins': row['wins'],
                'overall_losses': row['losses'],
                'rank': row['rank'],
                'predictive_score': predictive_scores.get(team, 0.0)
            })
        
        conf_standings_df = pd.DataFrame(conf_standings)
        
        # Sort by: Conference Win% (desc), Overall Record (desc), Composite Rank (asc)
        conf_standings_df = conf_standings_df.sort_values(
            by=['conf_win_pct', 'overall_wins', 'rank'],
            ascending=[False, False, True]
        ).reset_index(drop=True)
        
        # Get top 2 teams for championship game
        if len(conf_standings_df) >= 2:
            team_a_row = conf_standings_df.iloc[0]  # Higher seed (better record/rank)
            team_b_row = conf_standings_df.iloc[1]  # Lower seed
            
            team_a = team_a_row['team']
            team_b = team_b_row['team']
            team_a_score = team_a_row['predictive_score']
            team_b_score = team_b_row['predictive_score']
            
            # Simulate the championship game
            # Add Home Field Advantage for higher seed (neutral site but slight advantage)
            # Championship games are usually neutral but higher seed gets slight edge
            hfa_adjustment = 0.02  # Small HFA (~3 points in normalized score)
            adjusted_team_a_score = team_a_score + hfa_adjustment
            
            # Calculate spread (normalized score difference)
            spread = adjusted_team_a_score - team_b_score
            
            # Deterministic method: Higher predictive score wins
            # (Uncomment probabilistic method below for upset simulation)
            if spread > 0:
                winner = team_a
                winner_seed = 1
                loser = team_b
            else:
                winner = team_b
                winner_seed = 2
                loser = team_a
            
            # Probabilistic method (allows upsets) - uncomment to enable:
            # import random
            # win_prob = 1 / (1 + 10 ** (-spread / 0.17))  # Logistic function
            # if random.random() < win_prob:
            #     winner = team_a
            #     winner_seed = 1
            #     loser = team_b
            # else:
            #     winner = team_b
            #     winner_seed = 2
            #     loser = team_a
            
            conf_champions[conf] = winner
            
            # Determine if upset occurred
            team_a_rank = team_a_row['rank']
            team_b_rank = team_b_row['rank']
            upset = "⚠️ UPSET!" if (winner_seed == 2 and team_b_rank > team_a_rank) else ""
            
            print(f"  {conf:<25} #{team_a_rank} {team_a:<20} vs #{team_b_rank} {team_b:<20} → {winner} {upset}")
            
        elif len(conf_standings_df) == 1:
            # Edge case: Only one team in conference
            conf_champions[conf] = conf_standings_df.iloc[0]['team']
            print(f"  {conf:<25} Only one team: {conf_champions[conf]}")
    
    print()

print("✅ Conference champions identified")
if conf_champions:
    print("\nConference Champions:")
    for conf, team in sorted(conf_champions.items()):
        if championship_games_found:
            status = "✅ (from game data)"
        else:
            status = "🎮 (simulated)"
        print(f"  {conf:<20} {team:<25} {status}")
else:
    print("    (No conference champions identified)")

🎮 Simulating scheduled conference championship games...

  Sun Belt                  #3 James Madison        vs #60 Troy                 → James Madison
  Conference USA            #59 Jacksonville State   vs #31 Kennesaw State       → Kennesaw State
  Mountain West             #25 Boise State          vs #53 UNLV                 → Boise State
  American Athletic         #28 Tulane               vs #13 North Texas          → North Texas
  Mid-American              #44 Western Michigan     vs #24 Miami (OH)           → Miami (OH)
  Big 12                    #8 Texas Tech           vs #9 BYU                  → Texas Tech
  SEC                       #2 Alabama              vs #6 Georgia              → Georgia ⚠️ UPSET!
  ACC                       #16 Virginia             vs #29 Duke                 → Duke ⚠️ UPSET!
  Big Ten                   #4 Ohio State           vs #7 Indiana              → Ohio State

✅ Conference champions identified

Conference Champions:
  ACC                  Duk

---

## Resume Team Sheets

Complete resume view for all teams, showing:
- Rank and record
- SOR rank (how impressive is the record?)
- SOS rank (how tough was the schedule?)
- Performance vs Top 25 (W-L)
- Bad losses count
- Conference champion status

In [7]:
# Cell 7: Build Resume DataFrame
# Prepare data for resume builder
teams_data = final_rankings[['team', 'wins', 'losses']].copy()

# Add conference information from games data
if 'conference' not in final_rankings.columns:
    # Extract conference from games dataframe if not in final_rankings
    team_conferences = {}
    for _, game in games_df.iterrows():
        if pd.notna(game.get('home_conference')):
            team_conferences[game['home_team']] = game['home_conference']
        if pd.notna(game.get('away_conference')):
            team_conferences[game['away_team']] = game['away_conference']
    
    # Map conferences to teams_data
    teams_data['conference'] = teams_data['team'].map(team_conferences)
else:
    teams_data['conference'] = final_rankings['conference']

# Build composite ranks dict
composite_ranks = dict(zip(final_rankings['team'], final_rankings['rank']))

# Build resume DataFrame
resume_df = build_resume_dataframe(
    teams_data=teams_data,
    sor_scores=sor_scores,
    sos_scores=sos_scores,
    quality_wins=quality_wins_data,
    bad_losses=bad_losses_data,
    conf_champions=conf_champions,
    composite_ranks=composite_ranks
)

# Add vs_top_25 column with our calculated data
resume_df['vs_top_25'] = resume_df['team'].map(vs_top25_data)

# Save resume rankings
output_path = rankings_dir / f'resume_rankings_{year}_week{week}.csv'
resume_df.to_csv(output_path, index=False)

print(f"✅ Resume rankings saved to {output_path}")
print("\n" + "=" * 100)
print("TOP 25 RESUME TEAM SHEETS".center(100))
print("=" * 100)
print()

# Display top 25
display_cols = ['rank', 'team', 'record', 'sor_rank', 'sos_rank', 'vs_top_25', 'bad_losses', 'conf_champ']
resume_df[display_cols].head(25)

✅ Resume rankings saved to data/output/rankings/resume_rankings_2025_week15.csv

                                     TOP 25 RESUME TEAM SHEETS                                      



,rank,team,record,sor_rank,sos_rank,vs_top_25,bad_losses,conf_champ
0,1,Notre Dame,9-0,1,94,2-0,0,No
1,2,Alabama,7-1,4,13,2-1,0,No
2,3,James Madison,9-0,2,75,1-0,0,Yes (Sun Belt)
3,4,Ohio State,9-0,7,112,0-0,0,Yes (Big Ten)
4,5,Oregon,7-1,11,22,1-1,0,No
5,6,Georgia,8-1,12,64,2-1,0,Yes (SEC)
6,7,Indiana,8-0,5,134,1-0,0,No
7,8,Texas Tech,7-1,23,89,1-0,1,Yes (Big 12)
8,9,BYU,8-1,9,39,1-1,0,No
9,10,Utah,7-1,19,61,0-1,0,No


---

## Resume vs Predictive Comparison

This section separates **deserving** (resume) from **best** (predictive):

**Resume Rankings** (what you've accomplished):
- Based on: Colley Matrix (win/loss only), Win %, SOR
- No margin of victory
- Rewards quality wins and tough schedules

**Predictive Rankings** (how good you are):
- Based on: Massey Ratings (MOV), Elo, efficiency metrics
- Uses capped margin of victory (28 points)
- HFA-adjusted
- Predicts future game outcomes

**Composite Rankings:**
- Weighted blend of resume and predictive
- Balances accomplishment with power

In [8]:
# Cell 8: Resume vs Predictive Split
from sklearn.preprocessing import MinMaxScaler

# Load component rankings to reconstruct resume and predictive scores
# Note: final_rankings (composite_rankings.csv) doesn't have colley_rating, win_pct, massey_rating, elo_rating
# So we need to load the component files
win_pct_rankings = pd.read_csv(rankings_dir / f'win_pct_rankings_{year}_week{week}.csv')

# Build resume-only rankings (Colley + Win% + SOR)
resume_data = colley_rankings.merge(
    win_pct_rankings[['team', 'wins', 'losses', 'win_pct']],
    on='team'
)
resume_data['sor_score'] = resume_data['team'].map(sor_scores)

# Normalize resume components
scaler_resume = MinMaxScaler()
resume_normalized = resume_data.copy()
resume_normalized[['colley_rating', 'win_pct', 'sor_score']] = scaler_resume.fit_transform(
    resume_data[['colley_rating', 'win_pct', 'sor_score']]
)

# SOR is already in correct direction (higher sor_score = harder achievement = better)
# No inversion needed - sor_score = -log10(prob), so higher is better

# Calculate resume score (equal weights)
resume_normalized['resume_score'] = (
    0.40 * resume_normalized['colley_rating'] +
    0.30 * resume_normalized['win_pct'] +
    0.30 * resume_normalized['sor_score']
)

resume_normalized['resume_rank'] = resume_normalized['resume_score'].rank(
    method='min', ascending=False
).astype(int)

# Build predictive-only rankings (Massey + Elo)
predictive_data = massey_rankings.merge(
    elo_rankings[['team', 'elo_rating']],
    on='team'
)

# Normalize predictive components
scaler_predictive = MinMaxScaler()
predictive_normalized = predictive_data.copy()
predictive_normalized[['massey_rating', 'elo_rating']] = scaler_predictive.fit_transform(
    predictive_data[['massey_rating', 'elo_rating']]
)

# Calculate predictive score
predictive_normalized['predictive_score'] = (
    0.55 * predictive_normalized['massey_rating'] +
    0.45 * predictive_normalized['elo_rating']
)

predictive_normalized['predictive_rank'] = predictive_normalized['predictive_score'].rank(
    method='min', ascending=False
).astype(int)

# Create comparison dataframe
resume_ranks_dict = dict(zip(resume_normalized['team'], resume_normalized['resume_rank']))
predictive_ranks_dict = dict(zip(predictive_normalized['team'], predictive_normalized['predictive_rank']))
composite_scores_dict = dict(zip(final_rankings['team'], final_rankings['composite_score']))

comparison_df = compare_resume_vs_predictive(
    resume_ranks=resume_ranks_dict,
    predictive_ranks=predictive_ranks_dict,
    composite_ranks=composite_ranks,
    composite_scores=composite_scores_dict,
    top_n=25
)

# Add delta columns
comparison_df['resume_vs_composite'] = comparison_df['resume_rank'] - comparison_df['composite_rank']
comparison_df['predictive_vs_composite'] = comparison_df['predictive_rank'] - comparison_df['composite_rank']

# Save comparison
comparison_path = exports_dir / f'resume_vs_predictive_{year}_week{week}.csv'
comparison_df.to_csv(comparison_path, index=False)

print(f"✅ Comparison saved to {comparison_path}")
print("\n" + "=" * 100)
print("RESUME VS PREDICTIVE COMPARISON - TOP 25".center(100))
print("=" * 100)
print()
print("Legend:")
print("  Resume Rank:     Based on what teams have accomplished (wins, SOR, schedule)")
print("  Predictive Rank: Based on how good teams are (power ratings, efficiency)")
print("  Composite Rank:  Balanced combination of both")
print()

comparison_df[[
    'team', 'resume_rank', 'predictive_rank', 'composite_rank',
    'resume_vs_composite', 'predictive_vs_composite'
]]


✅ Comparison saved to data/output/exports/resume_vs_predictive_2025_week15.csv

                              RESUME VS PREDICTIVE COMPARISON - TOP 25                              

Legend:
  Resume Rank:     Based on what teams have accomplished (wins, SOR, schedule)
  Predictive Rank: Based on how good teams are (power ratings, efficiency)
  Composite Rank:  Balanced combination of both



,team,resume_rank,predictive_rank,composite_rank,resume_vs_composite,predictive_vs_composite
0,Notre Dame,1,4,1,0,3
1,Alabama,4,20,2,2,18
2,James Madison,2,18,3,-1,15
3,Ohio State,5,7,4,1,3
4,Oregon,10,12,5,5,7
5,Georgia,9,17,6,3,11
6,Indiana,6,22,7,-1,15
7,Texas Tech,16,3,8,8,-5
8,BYU,7,42,9,-2,33
9,Utah,14,15,10,4,5


---

## Biggest Resume vs Predictive Discrepancies

Teams where resume and predictive rankings differ significantly:

In [9]:
# Cell 9: Analyze Discrepancies
comparison_full = comparison_df.copy()
comparison_full['resume_pred_diff'] = abs(
    comparison_full['resume_rank'] - comparison_full['predictive_rank']
)

biggest_discrepancies = comparison_full.nlargest(10, 'resume_pred_diff')

print("Teams with Biggest Resume vs Predictive Discrepancies:")
print("=" * 100)
print()

for _, row in biggest_discrepancies.iterrows():
    team = row['team']
    resume_rank = int(row['resume_rank'])
    pred_rank = int(row['predictive_rank'])
    comp_rank = int(row['composite_rank'])
    diff = int(row['resume_pred_diff'])
    
    # Determine if resume-favored or predictive-favored
    if resume_rank < pred_rank:
        favor = "RESUME-FAVORED"
        reason = "Strong wins/schedule but lower power rating"
    else:
        favor = "PREDICTIVE-FAVORED"
        reason = "High power rating but weaker resume"
    
    print(f"{team}")
    print(f"  Status: {favor}")
    print(f"  Resume Rank: #{resume_rank}")
    print(f"  Predictive Rank: #{pred_rank}")
    print(f"  Composite Rank: #{comp_rank}")
    print(f"  Difference: {diff} spots")
    print(f"  Why: {reason}")
    print()

Teams with Biggest Resume vs Predictive Discrepancies:

Washington State
  Status: PREDICTIVE-FAVORED
  Resume Rank: #70
  Predictive Rank: #2
  Composite Rank: #19
  Difference: 68 spots
  Why: High power rating but weaker resume

Ole Miss
  Status: RESUME-FAVORED
  Resume Rank: #3
  Predictive Rank: #59
  Composite Rank: #11
  Difference: 56 spots
  Why: Strong wins/schedule but lower power rating

Virginia
  Status: RESUME-FAVORED
  Resume Rank: #11
  Predictive Rank: #52
  Composite Rank: #16
  Difference: 41 spots
  Why: Strong wins/schedule but lower power rating

BYU
  Status: RESUME-FAVORED
  Resume Rank: #7
  Predictive Rank: #42
  Composite Rank: #9
  Difference: 35 spots
  Why: Strong wins/schedule but lower power rating

Texas
  Status: RESUME-FAVORED
  Resume Rank: #21
  Predictive Rank: #54
  Composite Rank: #22
  Difference: 33 spots
  Why: Strong wins/schedule but lower power rating

Boise State
  Status: PREDICTIVE-FAVORED
  Resume Rank: #39
  Predictive Rank: #10
  Co

---

## Export Team Sheets

Export detailed team sheets for external use.

In [10]:
# Cell 10: Export Team Sheets
# Combine resume data with quality wins detail
team_sheets = resume_df.copy()

# Add quality wins breakdown
team_sheets['top_5_wins'] = team_sheets['team'].apply(
    lambda t: quality_wins_data.get(t, {}).get('top_5', 0)
)
team_sheets['top_12_wins'] = team_sheets['team'].apply(
    lambda t: quality_wins_data.get(t, {}).get('top_12', 0)
)
team_sheets['top_25_wins'] = team_sheets['team'].apply(
    lambda t: quality_wins_data.get(t, {}).get('top_25', 0)
)

# Add resume vs predictive ranks
team_sheets['resume_rank'] = team_sheets['team'].map(resume_ranks_dict)
team_sheets['predictive_rank'] = team_sheets['team'].map(predictive_ranks_dict)

# Export to CSV
team_sheets_path = exports_dir / f'team_sheets_{year}_week{week}.csv'
team_sheets.to_csv(team_sheets_path, index=False)

# Export top 25 to JSON for external consumption
top25_json = team_sheets.head(25).to_dict('records')
json_path = exports_dir / f'top25_team_sheets_{year}_week{week}.json'
with open(json_path, 'w') as f:
    json.dump(top25_json, f, indent=2)

print(f"✅ Team sheets exported:")
print(f"   CSV: {team_sheets_path}")
print(f"   JSON: {json_path}")
print()
print("Summary Statistics:")
print(f"  Total teams analyzed: {len(team_sheets)}")
print(f"  Conference champions: {len(conf_champions)}")
print(f"  Teams with Top 25 wins: {len([t for t, qw in quality_wins_data.items() if qw.get('top_25', 0) > 0])}")
print(f"  Teams with bad losses: {len([t for t, bl in bad_losses_data.items() if bl > 0])}")

✅ Team sheets exported:
   CSV: data/output/exports/team_sheets_2025_week15.csv
   JSON: data/output/exports/top25_team_sheets_2025_week15.json

Summary Statistics:
  Total teams analyzed: 136
  Conference champions: 9
  Teams with Top 25 wins: 30
  Teams with bad losses: 117


---

## Summary

Resume analysis complete. Key outputs:

1. **Resume Rankings** - Teams ranked by accomplishments (SOR, SOS, quality wins)
2. **Team Sheets** - Complete resume view for all teams
3. **Resume vs Predictive** - Clear separation of deserving vs best
4. **Quality Metrics** - Detailed breakdown of wins and losses

Next step: Proceed to **05_playoff_selection.ipynb** for 12-team bracket selection.